# Test Connection
Make sure to copy the `.env-template` to `.env` and fill in your Aleph Alpha API token.

In [2]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

True

In [3]:
from intelligence_layer.core import (
    CompleteInput,
    InMemoryTracer, Llama3InstructModel,
)

tracer = InMemoryTracer()

model = Llama3InstructModel(name="pharia-1-llm-7b-control")

prompt = model.to_instruct_prompt(
    instruction="Du bist ein Assistent, der Fragen beantwortet.",
    input="Wann wurde Köln gegründet?",
)
complete_input = CompleteInput(prompt=prompt)
response = model.complete(complete_input, tracer=tracer)
response

/Users/jens.luecke/Library/Caches/pypoetry/virtualenvs/rag-qa-workshop-xsC4sNM2-py3.12/lib/python3.12/site-packages/aleph_alpha_client/completion.py:9: SyntaxWarning: invalid escape sequence '\,'
  """
/Users/jens.luecke/Library/Caches/pypoetry/virtualenvs/rag-qa-workshop-xsC4sNM2-py3.12/lib/python3.12/site-packages/intelligence_layer/core/model.py:229: UserWarning: The provided model is not a recommended model for this model class.Make sure that the model you have selected is suited to be use for the prompt template used in this model class.
  warnings.warn(


No CLIENT_URL specified in environment, using default: https://api.aleph-alpha.com.


CompleteOutput(model_version='.unknown.', completions=[CompletionResult(log_probs=None, completion=' Köln wurde im Jahr 38 v. Chr. von den Römern gegründet und trägt seitdem den offiziellen Namen Colonia Claudia Ara Agrippinensium, abgekürzt CCAA.', completion_tokens=None, finish_reason='end_of_text', raw_completion=None)], num_tokens_prompt_total=35, num_tokens_generated=41, optimized_prompt=None)

# Atman for Source Highlighting
1. Run the instruction-input tuple below with the previously instantiated model.
2. Create a `TextHighlight` task with the same model and run it on the same prompt and the generated output.
3. Output the found highlights ordered by the score.
4. Change the wording in `user_input` from "spannend" to "langweilig", rerun and see what happens.

In [5]:
from intelligence_layer.core import (
    TextHighlight,
    TextHighlightInput,
)
from aleph_alpha_client import PromptGranularity

In [6]:
instruction = (
    "Sie sind ein Assistent, der Filmkritiken klassifiziert. Sie müssen entscheiden, ob die Person den Film mochte oder nicht."
    + ' Die Ausgabe sollte nur aus den Labels "mögen" oder "nicht mögen" bestehen.'
)
user_input = (
    "Nachricht: Ich habe gestern einen Film geschaut. Es ging um einen Agenten, der eine Mission erfüllen musste. Es war wirklich spannend."
    + " Zunächst musste der Agent eine Basis infiltrieren. Dort gab es einen Bösewicht. Blablabla. Lorem Ipsum dolor."
)